In [1]:
# 코드 3-14 보스턴 주택 데이터셋 로드하기
from keras.datasets import boston_housing

(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

Using TensorFlow backend.


In [2]:
# 코드 3-25 데이터 정규화하기
mean = train_data.mean(axis = 0)
train_data -= mean
std = train_data.std(axis = 0)
train_data /= std

test_data -= mean
test_data /= std

In [3]:
# 코드 3-26 모델 정의하기
from keras import models
from keras import layers

def build_model(): # 동일한 모델을 여러 번 생성할 것이므로 함수를 만들어 사용합니다.
    model = models.Sequential()
    model.add(layers.Dense(64, activation = "relu", input_shape = (train_data.shape[1],)))
    model.add(layers.Dense(64, activation = "relu"))
    model.add(layers.Dense(1))
    
    model.compile(loss = "mse",
                 optimizer="rmsprop",
                 metrics = ["mae"])
    return model

In [ ]:
# 코드 3-27 K-겹 검증하기
import numpy as np

k = 4
num_val_samples = len(train_data) // k
num_epochs = 100
all_scores = []

for i in range(k):
    print("처리중인 폴드 #", i)
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples] # 검증 데이터 준비: k번째 분할
    val_targets = train_targets[i * num_val_samples: (i + 1)* num_val_samples]
    
    partial_train_data = np.concatenate([train_data[:i * num_val_samples], # 훈련 데이터 준비: 다른 분할 전체
                                        train_data[(i + 1) * num_val_samples:]], axis = 0)
    
    partial_train_targets = np.concatenate([train_targets[:i * num_val_samples],
                                           train_targets[(i + 1) * num_val_samples:]], axis = 0)
    
    model = build_model()
    model.fit(partial_train_data, partial_train_targets, epochs = num_epochs, batch_size =1, verbose =0)
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose = 0)
    all_scores.append(val_mae)

처리중인 폴드 # 0
처리중인 폴드 # 1


In [ ]:
all_scores

In [ ]:
np.mean(all_scores)

In [ ]:
# 코드 3-28 각 폴드에서 검증 점수를 로그에 저장하기
num_epochs = 500
all_mae_histories = []

for i in range(k):
    print("처리중인 폴드 #", i)
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples] # 검증 데이터 준비: k번째 분할
    val_targets = train_targets[i * num_val_samples: (i + 1)* num_val_samples]
    
    partial_train_data = np.concatenate([train_data[:i * num_val_samples], # 훈련 데이터 준비: 다른 분할 전체
                                        train_data[(i + 1) * num_val_samples:]], axis = 0)
    
    partial_train_targets = np.concatenate([train_targets[:i * num_val_samples],
                                           train_targets[(i + 1) * num_val_samples:]], axis = 0)
    
    model = build_model()
    history = model.fit(partial_train_data, partial_train_targets, validation_data = (val_data, val_targets),
              epochs = num_epochs, batch_size =1, verbose =0)
    mae_history = history.history["val_mae"]
    all_mae_histories.append(mae_history)

In [ ]:
# 코드 3-29 K-겹 검증 점수 평균을 기록하기
average_mae_history = [
    np.mean([x[i] for x in all_mae_histories]) for i in range(num_epochs)]

In [ ]:
# 코드 3-30 검증 점수 그래프
import matplotlib.pyplot as plt

plt.plot(range(1, len(average_mae_history) + 1), average_mae_history)
plt.xlabel("Epochs")
plt.ylabel("Validation MAE")
plt.show()

In [ ]:
# 코드 3-31 처음 10개의 데이터 포인트를 제외한 검증 점수 그리기
def smooth_curve(points, factor = 0.9):
    smoothed_points = []
    for point in points:
        if smoothed_points:
            previous = smoothed_points[-1]
            smoothed_points.append(previous * factor + point * (1 - factor))
        else :
            smoothed_points.append(point)
    return smoothed_points

smooth_mae_history = smooth_curve(average_mae_history[10:])

plt.plot(range(1, len(smooth_mae_history) + 1), smooth_mae_history)
plt.xlabel("Epochs")
plt.ylable("Validation MAE")
plt.show()    

In [ ]:
# 코드 3-32 최종 모델 훈련하기
model = build_model()
model.fit(train_data, train_targets, epochs = 80, batch_size = 16, verbose = 0)
test_mse_score, test_mae_score = model.evaluate(test_data, test_targets)

In [ ]:
test_mae_score